In [342]:
import math
import random
import sys
from math import ceil, floor
import numpy as np
import pandas as pd

swapTime = 600      #seconds per vehicle
GridX = 10000       #meters
GridY = 10000       #meters
intervalOfChargingStations = 2000       #meters
noOfRequests = 150     #per hour
numberOfChargingStations = GridX*GridY//intervalOfChargingStations**2
int_min_speed=20    #kmph
int_max_speed=40    #kmph
high_min_speed=60   #kmph
high_max_speed=80   #kmph
startTime = 0

################################################################################################################

ChargingStationCode = []
CSx = []
CSy = []
occupancyTime=[]
averageActulTime=[]
noOfCars=[]

randomTime=[]
randomX=[]
randomY=[]
allocatedCS=[]
arrivalTime=[]
allotCS = 0
ans = []
aans = []
waitans = []


In [343]:
def makeStations(GridX,GridY,intervalOfChargingStations):
    for i in range(0,GridX,intervalOfChargingStations):
        for j in range(0,GridY,intervalOfChargingStations):
            name = "CS"+str(i//1000)+str(j//1000)
            ChargingStationCode.append(name)
            CSx.append(i)
            CSy.append(j)
            occupancyTime.append(0)
            averageActulTime.append(0)
            noOfCars.append(0)

In [344]:
def distanceBetweenAB(A,B):
    hor_dis=abs(A[0]-B[0])
    ver_dis=abs(A[1]-B[1])
    return hor_dis + ver_dis

In [345]:
def findNearestHighway(A,B):
    A1 = []
    x1 = A[0]
    y1 = A[1]
    x2 = B[0]
    y2 = B[1]
    if x2>x1:
        vert_highway=ceil(x1/1000)*1000
    else:
        vert_highway=floor(x1/1000)*1000
    if y2>y1:
        horiz_highway=ceil(y1/1000)*1000
    else:
        horiz_highway=floor(y1/1000)*1000
    horHigPointA=[x1,horiz_highway]
    verHigPointA=[vert_highway,y1]

    #for A
    if distanceBetweenAB(A,horHigPointA) < distanceBetweenAB(A,verHigPointA):
        A1.append(x1)
        A1.append(horiz_highway)
    else:
        A1.append(vert_highway)
        A1.append(y1)
    return A1

In [346]:
def timeToTravel(A,B):
    Nearest_Highway_pointA = findNearestHighway(A,B)[:]
    high_distance = distanceBetweenAB(Nearest_Highway_pointA,B)
    int_distanceA = distanceBetweenAB(A,Nearest_Highway_pointA)
    highway_speed = (random.randint(high_min_speed,high_max_speed)*5)/18
    interior_speed = (random.randint(int_min_speed,int_max_speed)*5)/18
    high_time = high_distance/highway_speed
    inter_time = int_distanceA/interior_speed
    travel_time = high_time + inter_time
    travel_time  = round(travel_time,2)
    return travel_time

In [347]:
makeStations(GridX, GridY, intervalOfChargingStations)


In [348]:
def waitingTime(rax, tmp, x, allotcs):
    ans = 0
    if(x < rax+tmp):
        ans = 0
        occupancyTime[allotcs] += 600
    else:
        ans = x-rax-tmp
        occupancyTime[allotcs] += 600
    return ans

In [349]:
def makeZero(array):
    for i in range(len(array)):
        array[i]=0


In [350]:
def occupancyTimeRemaining(array):
    for i in range(len(array)):
        if(array[i]>3600):
            array[i] = array[i]-3600
        else:
            array[i] = 0

In [351]:
def restart():
    global randomTime 
    randomTime=[]
    global randomX
    randomX=[]
    global randomY
    randomY=[]
    global allocatedCS
    allocatedCS=[]
    global arrivalTime
    arrivalTime=[]


In [352]:
def delta(n):
    return n//10

In [353]:

def vehicleDensity(time):
    if time >= 0 and time < 6:
        return noOfRequests - 2 * delta(noOfRequests)
    elif time >= 6 and time < 8:
        return noOfRequests - delta(noOfRequests)
    elif time >= 8 and time < 11:
        return noOfRequests + delta(noOfRequests)
    elif time >=11 and time < 13:
        return noOfRequests
    elif time >=13 and time < 16:
        return noOfRequests - delta(noOfRequests)
    elif time >=16 and time < 18:
        return noOfRequests
    elif time >=18 and time < 21:
        return noOfRequests + delta(noOfRequests)
    else:
        return noOfRequests - delta(noOfRequests)


In [354]:
df = pd.read_csv('randomData.csv')

In [355]:
sams = []
distancewise = []
waitingwise = []
# finalans = []

In [356]:
def solveRequest(requests):
    for i in range(requests):
        X = [randomX[i], randomY[i]]
        tmp = sys.maxsize
        allotcs = 0
        for j in range(numberOfChargingStations):
            Y = [CSx[j], CSy[j]]
            dist = timeToTravel(X, Y)
            if(dist<tmp):
                tmp = dist
                allotcs = j
        wait = waitingTime(randomTime[i], tmp, occupancyTime[allotcs], allotcs)
        Y = [CSx[allotcs], CSy[allotcs]]
        vx = distanceBetweenAB(X,Y)
        distancewise.append(vx)
        waitingwise.append(wait)
        sams.append(wait+tmp)
        
        
    return sams
        
        


In [358]:

for x in range(24):
    
    requests = vehicleDensity(x)
    m = 'randomTime'+str(x)+'(seconds)'
    n = 'randomX'+str(x)+'(meters)'
    o = 'randomY'+str(x)+'(meters)'
    randomTime = df[m].tolist()[:requests]
    randomX = df[n].tolist()[:requests]
    randomY = df[o].tolist()[:requests]
    xx = solveRequest(requests)
    # print(xx)
    # print("fcuk")
    ans.append(xx)
    aans.append(distancewise)
    waitans.append(waitingwise)
    # print(sams)
    # print(len(sams))
    # print(ans)
    restart()
    sams = []
    distancewise = []
    waitingwise = []
    
    


In [359]:
# for x in range(24):
    
#     requests = vehicleDensity(x)
#     m = 'randomTime'+str(x)+'(seconds)'
#     n = 'randomX'+str(x)+'(meters)'
#     o = 'randomY'+str(x)+'(meters)'
#     randomTime = df[m].tolist()[:requests]
#     randomX = df[n].tolist()[:requests]
#     randomY = df[o].tolist()[:requests]
#     xx = solveRequestd(requests)
#     # print(xx)
#     # print("fcuk")
#     aans.append(xx)
#     # print(sams)
#     # print(len(sams))
#     # print(ans)
#     restart()
#     distancewise = []


In [360]:
df69 = pd.DataFrame({'lastRow': ans})
df69.to_csv('lastRow.csv', index=False)


In [361]:
df70 = pd.DataFrame({'distRow': aans})
df70.to_csv('distRow.csv', index=False)

In [ ]:
df71 = pd.DataFrame({'waittimewise': waitans})
df71.to_csv('waittimewise.csv', index=False)